In [6]:
import pandas as pd

#function to convert table to markdown
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)

### For each game, computed weighted average of BA/AB called ABBA

In [2]:
df = pd.read_csv("../no_hits_res.csv", encoding = "ISO-8859-1")
df.date = df.date.astype(str).str[:10]
dfi = df[['Pitcher','date','Team','Opponent','game_link']].drop_duplicates().reset_index(drop=1).reset_index()
dfi.columns=['id','Pitcher','date','Team','Opponent','game_link']

df = df.merge(dfi)
df['abba']=df.AB*df.BA

dfs = df.groupby('id')['AB','abba'].sum()
dfs['avg'] = dfs.abba / dfs.AB
dfo = dfs.reset_index().merge(dfi).sort_values('avg', ascending=0).reset_index(drop=True)


In [3]:
#function for computing relative odds
def calculate_odds(b, a=0.250):
    if a >=b:
        r = f"{round(( (1-b)**27 ) / ( (1-a)**27),2)} times more likely"
    else:
        r = f"{round(( (1-a)**27 ) / ( (1-b)**27),2)} times less likely"
    
    return r



In [4]:
#relative odds and rank
dfo['Odds'] = dfo.avg.apply(calculate_odds)
dfo['Rank'] = dfo.avg.rank(ascending=False).astype(int)

In [5]:
#dfop is formatted for output 
dfop = dfo[['Rank','Pitcher','date','Team','Opponent','avg','Odds','game_link']].copy()
dfop.columns = ['Rank','Pitcher','Date','Team','Opponent','ABBA','Odds vs 0.250 ABBA','Boxscore']
dfop.Boxscore = dfop.Boxscore.apply(lambda x: f"[Link]({x})")
dfop.ABBA = round(dfop.ABBA, 4)

In [7]:
#top 15
print(pandas_df_to_markdown_table(dfop.head(15)))

Rank|Pitcher|Date|Team|Opponent|ABBA|Odds vs 0.250 ABBA|Boxscore
---|---|---|---|---|---|---|---
1|Hideo Nomo (1)|1996-09-17|Los Angeles Dodgers|Colorado Rockies|0.2899|4.38 times less likely|[Link](https://www.baseball-reference.com/boxes/COL/COL199609170.shtml)
2|Dwight Gooden|1996-05-14|New York Yankees|Seattle Mariners|0.2878|4.05 times less likely|[Link](https://www.baseball-reference.com/boxes/NYA/NYA199605140.shtml)
3|Vern Bickford|1950-08-11|Boston Braves|Brooklyn Dodgers|0.2858|3.74 times less likely|[Link](https://www.baseball-reference.com/boxes/BSN/BSN195008110.shtml)
4|Al Leiter|1996-05-11|Florida Marlins|Colorado Rockies|0.2792|2.92 times less likely|[Link](https://www.baseball-reference.com/boxes/FLO/FLO199605110.shtml)
5|Roy Halladay (2)|2010-10-06|Philadelphia Phillies|Cincinnati Reds|0.277|2.69 times less likely|[Link](https://www.baseball-reference.com/boxes/PHI/PHI201010060.shtml)
6|David Cone|1999-07-18|New York Yankees (AL)|Montreal Expos (NL)|0.2767|2.66 times le

In [8]:
#bottom 15
print(pandas_df_to_markdown_table(dfop.tail(15)))

Rank|Pitcher|Date|Team|Opponent|ABBA|Odds vs 0.250 ABBA|Boxscore
---|---|---|---|---|---|---|---
172|Homer Bailey (1)|2012-09-28|Cincinnati Reds|Pittsburgh Pirates|0.2319|1.9 times more likely|[Link](https://www.baseball-reference.com/boxes/PIT/PIT201209280.shtml)
173|Chris Heston|2015-06-09|San Francisco Giants|New York Mets|0.2318|1.91 times more likely|[Link](https://www.baseball-reference.com/boxes/NYN/NYN201506090.shtml)
174|Don Wilson (1)|1967-06-18|Houston Astros|Atlanta Braves|0.2316|1.92 times more likely|[Link](https://www.baseball-reference.com/boxes/HOU/HOU196706180.shtml)
175|Matt Cain|2012-06-13|San Francisco Giants|Houston Astros|0.2315|1.93 times more likely|[Link](https://www.baseball-reference.com/boxes/SFN/SFN201206130.shtml)
176|Bill Singer|1970-07-20|Los Angeles Dodgers|Philadelphia Phillies|0.2308|1.98 times more likely|[Link](https://www.baseball-reference.com/boxes/LAN/LAN197007200.shtml)
177|Tom Phoebus|1968-04-27|Baltimore Orioles|Boston Red Sox|0.2292|2.1 tim

In [9]:
#nolan ryan
ryan = dfop[dfop.Pitcher.str.contains("Nolan Ryan")]
print(pandas_df_to_markdown_table(ryan))

Rank|Pitcher|Date|Team|Opponent|ABBA|Odds vs 0.250 ABBA|Boxscore
---|---|---|---|---|---|---|---
16|Nolan Ryan (3)|1974-09-28|California Angels|Minnesota Twins|0.2704|2.11 times less likely|[Link](https://www.baseball-reference.com/boxes/CAL/CAL197409280.shtml)
39|Nolan Ryan (1)|1973-05-15|California Angels|Kansas City Royals|0.2648|1.71 times less likely|[Link](https://www.baseball-reference.com/boxes/KCA/KCA197305150.shtml)
42|Nolan Ryan (7)|1991-05-01|Texas Rangers|Toronto Blue Jays|0.2636|1.64 times less likely|[Link](https://www.baseball-reference.com/boxes/TEX/TEX199105010.shtml)
63|Nolan Ryan (5)|1981-09-26|Houston Astros|Los Angeles Dodgers|0.2595|1.41 times less likely|[Link](https://www.baseball-reference.com/boxes/HOU/HOU198109260.shtml)
79|Nolan Ryan (2)|1973-07-15|California Angels|Detroit Tigers|0.2555|1.22 times less likely|[Link](https://www.baseball-reference.com/boxes/DET/DET197307150.shtml)
85|Nolan Ryan (6)|1990-06-11|Texas Rangers|Oakland Athletics|0.2549|1.19 time